In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
test = pd.read_csv('/kaggle/input/cat-in-the-dat-ii/test.csv')
train = pd.read_csv('/kaggle/input/cat-in-the-dat-ii/train.csv')

In [ ]:
%%time

# Random permutation is needed for CatBoostEncoder to reduce leakage
def random_permutation(x):
    perm = np.random.permutation(len(x)) 
    x = x.iloc[perm].reset_index(drop=True) 
    return x

train = random_permutation(train)
test = random_permutation(test)

train_ids = train.id
test_ids = test.id

train.drop('id', 1, inplace=True)
test.drop('id', 1, inplace=True)

train_targets = train.target
train.drop('target', 1, inplace=True)

# Preprocessing

Preprocessing strategy:

* For high-cardinality features (`nom_5` to `nom_9` and `ord_5`) use `CatBoostEncoder` target encoding (it performs a type of leave-one-out encoding)
* For other features use `OneHotEncoder`

In [ ]:
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## Missing values

Convert columns to strings and replace NAs with a non-NaN value to ensure that the one-hot encoder will treat missing values as their own class.

In [ ]:
%%time

for col in train.columns:
    train[col] = train[col].astype(str)
    train[col].fillna('NA', inplace=True)
    test[col] = test[col].astype(str)
    test[col].fillna('NA', inplace=True)

## Encoders

### CatBoostEncoder for high-cardinality features

In [ ]:
%%time

# noms 5-9
for i in [5,6,7,8,9]:
    cbe = CatBoostEncoder()
    train[f'nom_{i}'] = cbe.fit_transform(train[f'nom_{i}'], train_targets)
    test[f'nom_{i}'] = cbe.transform(test[f'nom_{i}'])

# ord 5
cbe = CatBoostEncoder()
train['ord_5'] = cbe.fit_transform(train['ord_5'], train_targets)
test['ord_5'] = cbe.transform(test['ord_5'])

### OneHotEncoder for other features

In [ ]:
%%time

ohe_cols = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4',
            'nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4',
            'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4',
            'day', 'month']

# ColumnTransformer enables applying OneHotEncoder to the entire dataframe
transformer = ColumnTransformer(
    [
        ("ohe",
         OneHotEncoder(sparse=True, drop='first'),
         ohe_cols
         )
    ], remainder='passthrough'
)
train = transformer.fit_transform(train)
test = transformer.fit_transform(test)

# Logistic Regression

I'll use LogisticRegressionCV to do automatic parameter selection, and give me a sense of cross-validated model performance.

In [ ]:
%%time

from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, 
                           scoring='roc_auc', 
                           random_state=42, 
                           verbose=True, 
                           n_jobs=-1,
                           max_iter = 1000)
clf.fit(train, train_targets)

### Model score averaged over folds

In [ ]:
np.mean(clf.scores_[1])

# Prepare submission

In [ ]:
preds = clf.predict_proba(test)[:, 1]
preds = pd.DataFrame(list(zip(test_ids, preds)), columns = ['id', 'target'])
preds.sort_values(by=['id'], inplace = True)

preds.to_csv("./my_submission.csv", index=False)